In [2]:
from sift1m_dataset_builder import Builder
import tensorflow_datasets.public_api as tfds
from etils.epath import Path
import numpy as np
from sit1m_data_preprocessing import *
import time
import random

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import faiss
import torch

In [3]:
dataset_download_path = "D:/College/SysML/dataset"
splits = build_sift1m(dataset_download_path)

Building Sift1M


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:04<00:00,  4.64s/ url]
Extraction completed...: 0 file [00:04, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:04<00:00,  4.65s/ url]


In [4]:
train_input_array = get_train_split(splits)
test_input_array, neighbors = get_test_split(splits)

Building train split array
Found existing train data file
Building test split array
Progress: 50.0%
Progress: 100.0%


## Benchmark Evaluation

In [26]:
dim = train_input_array.shape[1]
numSubVectors = 8 #16
subVectorBits = 8

#model = faiss.IndexFlatL2(dim) # Flat
#model = faiss.IndexPQ(dim, numSubVectors, 8) # PQ8
#model = faiss.IndexPQ(dim, 16, 8) # PQ16
model = faiss.index_factory(dim, "IVF4096_HNSW,PQ32") # HNSW + PQ32


In [27]:
model.train(train_input_array)
model.add(train_input_array)

In [28]:
k = 5
distances, indexes = model.search(test_input_array[:10000], k)

In [29]:
# Relevancy

t = 25
i = 0
relevantPercent = []

for entry in neighbors:

    topIndexs = []
    for x in range(t):
        index_dict = entry[x]
        topIndexs.append(index_dict["index"])

    search_results = indexes[i]

    count = 0

    for x in search_results:
        if x in topIndexs:
            count+=1

    relevantPercent.append(count/k)
    
    i=i+1
        

print(sum(relevantPercent) / len(relevantPercent))

0.68914


In [30]:
# Size
idx_file = open("./temp.index", "w")
idx_file.truncate(0)
idx_file.close()
faiss.write_index(model, "./temp.index")
file_size = os.path.getsize('./temp.index')
os.remove("./temp.index")
print("File size in bytes: " + str(file_size))

File size in bytes: 43373033


In [31]:
# Search Speed
total_time = 0
iterations = 1000

for x in range(iterations):

    k = 25
    index = random.randint(0, 9999)

    input = np.empty((0,128))
    input = np.vstack([input,test_input_array[index]])

    start_time = time.time()
    distances, index = model.search(input, k)
    end_time = time.time()

    total_time += end_time - start_time


print("Average search time: ", total_time/iterations, " seconds")

Average search time:  0.00014315223693847656  seconds
